In [15]:
! pip install --q chromadb rank_bm25 pypdf

In [4]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00


In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [10]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [17]:
from langchain.document_loaders import PyPDFLoader
loader =  PyPDFLoader("/content/rag.pdf")
documents = loader.load()

In [18]:
documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with acmart 2020/02/22 v1.70 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2025-03-04T01:53:02+00:00', 'author': 'Shangyu Wu, Ying Xiong*, Yufei Cui, Haolun Wu, Can Chen, Ye Yuan, Lianming Huang, Xue Liu, Tei-Wei Kuo, Nan Guan, and Chun Jason Xue', 'keywords': '', 'moddate': '2025-03-04T01:53:02+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Retrieval-Augmented Generation for Natural Language Processing: A Survey', 'trapped': '/False', 'source': '/content/rag.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for Natural Language\nProcessing: A Survey\nShangyu Wu\nCity University of Hong Kong,\nMBZUAI\nYing Xiong*\nMBZUAI\nYufei Cui\nMcGill University, Mila\nHaolun Wu\nMcGill Univers

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [20]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)

In [21]:
retriever = vectorstore.as_retriever()

In [22]:
from langchain.retrievers import BM25Retriever
keyword_retriever = BM25Retriever.from_documents(documents)
keyword_retriever.k =  3

In [23]:
keyword_retriever.get_relevant_documents("what is rag?")

<ipython-input-23-c9d6bcbb937b>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  keyword_retriever.get_relevant_documents("what is rag?")


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with acmart 2020/02/22 v1.70 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2025-03-04T01:53:02+00:00', 'author': 'Shangyu Wu, Ying Xiong*, Yufei Cui, Haolun Wu, Can Chen, Ye Yuan, Lianming Huang, Xue Liu, Tei-Wei Kuo, Nan Guan, and Chun Jason Xue', 'keywords': '', 'moddate': '2025-03-04T01:53:02+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Retrieval-Augmented Generation for Natural Language Processing: A Survey', 'trapped': '/False', 'source': '/content/rag.pdf', 'total_pages': 19, 'page': 10, 'page_label': '11'}, page_content='generators in RAG can learn the pattern between questions and\nanswers and infer what answers should be. For some specific QA\ntasks where a set of reference documents is given, retrievers in RAG\nwould ret

In [25]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, keyword_retriever], weights=[0.5, 0.5])

In [26]:
ensemble_retriever.get_relevant_documents("what is rag")

[Document(metadata={'trapped': '/False', 'source': '/content/rag.pdf', 'producer': 'pdfTeX-1.40.25', 'keywords': '', 'title': 'Retrieval-Augmented Generation for Natural Language Processing: A Survey', 'moddate': '2025-03-04T01:53:02+00:00', 'total_pages': 19, 'page': 9, 'page_label': '10', 'subject': '', 'author': 'Shangyu Wu, Ying Xiong*, Yufei Cui, Haolun Wu, Can Chen, Ye Yuan, Lianming Huang, Xue Liu, Tei-Wei Kuo, Nan Guan, and Chun Jason Xue', 'creationdate': '2025-03-04T01:53:02+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2020/02/22 v1.70 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX'}, page_content='7 RAG EVALUATION AND BENCHMARK\nRetrieval-Augmented Generation (RAG) systems integrate the capa-\nbilities of large language models (LLMs) with external knowledge\nretrieval to advance text generation tasks 

In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [28]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the following context.
If you don't find the answer in the context, just say that you don't know.
Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": ensemble_retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
response = rag_chain.invoke('what is rag?')
response

'RAG stands for Retrieval-Augmented Generation. It refers to systems that integrate large language models with external knowledge retrieval to enhance text generation tasks in natural language processing (NLP).'

In [30]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langsmith import Client
client = Client()
prompt = client.pull_prompt("langchain-ai/rag-fusion-query-generation")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [37]:
generate_queries = (
    rag_chain|prompt | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
)

In [33]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [38]:
chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [39]:
chain.input_schema.schema()

{'title': 'RunnableParallel<context,input>Input', 'type': 'string'}

In [40]:
chain.invoke("what is rag?")

[(Document(metadata={'creator': 'LaTeX with acmart 2020/02/22 v1.70 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'source': '/content/rag.pdf', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'author': 'Shangyu Wu, Ying Xiong*, Yufei Cui, Haolun Wu, Can Chen, Ye Yuan, Lianming Huang, Xue Liu, Tei-Wei Kuo, Nan Guan, and Chun Jason Xue', 'subject': '', 'trapped': '/False', 'creationdate': '2025-03-04T01:53:02+00:00', 'total_pages': 19, 'page_label': '10', 'title': 'Retrieval-Augmented Generation for Natural Language Processing: A Survey', 'page': 9, 'producer': 'pdfTeX-1.40.25', 'keywords': '', 'moddate': '2025-03-04T01:53:02+00:00'}, page_content='7 RAG EVALUATION AND BENCHMARK\nRetrieval-Augmented Generation (RAG) systems integrate the capa-\nbilities of large language models (LLMs) with external knowledge\nretrieval to advance text generation tasks

In [41]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
If you don't find the answer in the context, just say that you don't know.

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_fusion_chain = (
    {
        "context": chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
rag_fusion_chain.invoke("what is rag and explain it's importance?")

'RAG stands for Retrieval-Augmented Generation. It is a system that combines large language models with external knowledge retrieval to enhance text generation tasks in natural language processing. RAG is important because it allows the system to effectively leverage external knowledge, produce accurate outputs, and ensure contextually appropriate responses by integrating external data into the generation process. It helps in making language models more knowledgeable, accurate, and reliable in various applications.'

In [45]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [46]:
from datasets import Dataset

data = {
    "question": ["What is RAG?"],
    "contexts": [[
        "RAG (Retrieval-Augmented Generation) is a method that combines retrieval-based and generation-based models. It retrieves relevant documents and then uses them to generate more accurate and grounded responses."
    ]],
    "answer": ["RAG is a method that retrieves documents and then generates responses based on them."],
    "ground_truth": ["RAG is a method that combines document retrieval with text generation to improve accuracy and grounding in responses."]
}

dataset = Dataset.from_dict(data)

In [48]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00


In [49]:
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'context_recall': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 1.0000}
